In [5]:
import numpy as np
import cv2
import scipy
import matplotlib.pylab as plt


In [7]:
video = cv2.VideoCapture('left_output.avi')       #ดึงวิดิโอ
template_img = cv2.imread('Template-1.png')      #ดึงรูปภาพ
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)    #เปลี่ยนBGR เป็นgray

sift = cv2.xfeatures2d.SIFT_create()
bf = cv2.BFMatcher()

Xx=0
Yy=0
Zz=0
width  = 0      
height = 0  


if video.isOpened(): 
    width  = video.get(3)  
    height = video.get(4)  # float height
    print('width, height:', width, height)
fps = video.get(cv2.CAP_PROP_FPS)
    # or
fps = video.get(5)   
print('fps:', fps)  # float `fps`
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # or
frame_count = video.get(7)
    
middle_W=width/2
middle_H=height/2

    
print('frames count:', frame_count)  # float `frame_count`

while video.isOpened() :
    ret, query_img = video.read()
    ret, img = video.read()
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

    if ret :
        
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)

        
        good_matches = list()
        good_matches_list = list()

        for m, n in matches :
            if m.distance < 0.58*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > 9.6 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            src_pts,dst_pts = np.float32((src_pts,dst_pts))
            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2.8) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)         
            transformed_box = cv2.perspectiveTransform(template_box, H)
            
            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)     #ตีกรอบ
            
            
            X0 = transformed_box[0][0]
            Y0 = transformed_box[1][0]
            Z0 = 0

            axis1=transformed_box[0]
            axis2=transformed_box[1]
            axissum=axis2[0]-axis1[0]
           
            
            cv2.putText(query_img, " X = {:.3f} m    Y = {:.3f} m    Z = {:.3f} m".format(float((X0[0]-middle_W)*0.000264),float((Y0[1]-middle_H)*0.000264), 
            float((axissum[1])*0.000264)),(10,query_img.shape[0] - 20), cv2.FONT_ITALIC,1, (0, 255, 255), 2)
            
    
            cv2.imshow('Video frame', detected_img)
          

        if cv2.waitKey(int(30)) & 0xFF == ord('q') : 
            break
    else :
        break

video.release()
cv2.destroyAllWindows()

width, height: 864.0 480.0
fps: 30.0
frames count: 1690.0
